[medium](https://medium.com/@jonathan_hui/rl-dqn-deep-q-network-e207751f7ae4) <br>
[ref](https://github.com/udacity/deep-reinforcement-learning/tree/master/dqn)

In [1]:
%reload_ext autoreload
%autoreload 2
import torch
import torch.nn.functional as F 
import random
import numpy as np
from EXITrl.approx_v_base import ApproxVBase
from EXITrl.approx_policy_base import ApproxPolicyBase
from EXITrl.base import Base
from EXITrl.helpers import print_weight_size, copy_params, update_params, ExperienceReplay, convert_to_tensor
from EXITrl.nn_wrapper import NNWrapper
import gym

In [2]:
class QNetwork(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.linear1 = torch.nn.Linear(input_size, hidden_size)
        self.linear2 = torch.nn.Linear(hidden_size, hidden_size)
        self.linear3 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        return self.linear3(x)

In [ ]:
class DQN(Base):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.local_q_network = NNWrapper(
            QNetwork(self.num_state, 64, self.num_action),
            lr=self.alpha
        )
        self.target_q_network = NNWrapper(
            QNetwork(self.num_state, 64, self.num_action),
            lr=self.beta
        )
        self.experience_replay = ExperienceReplay(num_experience=2048)
        
        self.update_count = 0
        self.epsilon = 1

    def _loop(self, episode) -> int:
        done = False
        total_reward, reward = 0, 0
        state = self.env.reset()
        for i in range(1000):
            self.epsilon *= 0.99
            if self.epsilon<0.01:
                self.epsilon= 0.01
            
            action = self.local_q_network.epsilon_greedy(state, self.epsilon)
            _state, reward, done, _ = self.env.step(action)
            self.experience_replay.remember(state, action, reward, _state, done)
            
            if self.update_count%4 == 0:
                batch_state, \
                batch_action, \
                batch_reward, \
                batch_next_state, \
                batch_done = self.experience_replay.recall(batch_size=64)

                # detach because we only backprop local network and update target network weight manually
                targets_next_Q = self.target_q_network.forward(batch_next_state).detach().max(1)[0]
                targets_Q = batch_reward + (self.gamma * targets_next_Q * (1 - batch_done))

                local_Q = self.local_q_network.forward(batch_state)
                expected_Q = local_Q.gather(1, batch_action.unsqueeze(1).long()).squeeze(1)

                loss = F.mse_loss(expected_Q, targets_Q)
                self.local_q_network.backprop(loss)
            
                update_params(self.local_q_network.model, self.target_q_network.model, self.tau)
            
            total_reward += reward
            state = _state
            self.update_count += 1
            if done: return total_reward
        
try: env.close()
except: pass
env = gym.make('LunarLander-v2')
dqn = DQN(env, 
      num_episodes=1000,
      alpha=0.0001, 
      beta=0.0001,
      gamma=.99)
dqn.train(True)

/Users/epinyoanun/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
episode: 0 reward: -175.6719309152444
episode: 1 reward: -274.2036314363211
episode: 2 reward: -722.2442052580515
episode: 3 reward: -604.2080585438855
episode: 4 reward: -609.0064950254724
episode: 5 reward: -746.7980859504426
episode: 6 reward: -368.5739171188871
episode: 7 reward: -361.50637711332973
episode: 8 reward: -300.7242017902653
episode: 9 reward: -213.75063227264351
episode: 10 reward: -392.0468522259577
episode: 11 reward: -423.35488025564035
episode: 12 reward: -384.03144561757983
episode: 13 reward: -339.97046435141135
episode: 14 reward: -257.6834502151773
episode: 15 reward: -74.38314981819939
episode: 16 reward: -178.16870332948685
episode: 17 reward: -160.72072009302877
episode: 18 reward: -194.29686943986513
episode: 19 reward: -211.77497581571183
episode: 20 reward: -182.33292643802736
episode: 21 reward: -129.6409757528953
episode: 22 reward: -209.4044546894741
epis

episode: 204 reward: -208.45953305789487
episode: 205 reward: -186.02351037636566
episode: 206 reward: -102.49398031014175
episode: 207 reward: -204.97813435975007
episode: 208 reward: -144.50872762729617
episode: 209 reward: -204.13124876808678
episode: 210 reward: -203.1463065344698
episode: 211 reward: -166.60191439819528
episode: 212 reward: -144.18963980850455
episode: 213 reward: -198.37004013383108
episode: 214 reward: -229.82572419729865
episode: 215 reward: -177.82667980689962
episode: 216 reward: -176.72956455991329
episode: 217 reward: -210.52785160041282
episode: 218 reward: -163.90683215380886
episode: 219 reward: -177.40116407154167
episode: 220 reward: -179.72572463067138
episode: 221 reward: -173.0306820535867
episode: 222 reward: -195.00359872345305
episode: 223 reward: -154.26997335706784
episode: 224 reward: -127.65625215225828
episode: 225 reward: -154.02885857354045
episode: 226 reward: -169.6159373520187
episode: 227 reward: -153.6750219204473
episode: 228 reward:

episode: 406 reward: -381.30473572750765
episode: 407 reward: -223.08556373090966
episode: 408 reward: -75.13335974147395
episode: 409 reward: -239.70892811441024
episode: 410 reward: -229.15229287615662
episode: 411 reward: -322.4740800261673
episode: 412 reward: -211.50681750886818
episode: 413 reward: -164.86452139128153
episode: 414 reward: -217.08654131743566
episode: 415 reward: -130.7270882541523
episode: 416 reward: -270.59072901730104
episode: 417 reward: -186.82526939782724
episode: 418 reward: -263.0587601512783
episode: 419 reward: -214.13598709533426
episode: 420 reward: -157.01027244109272
episode: 421 reward: -156.8989601187941
episode: 422 reward: -157.3788848955874
episode: 423 reward: -302.33723044978854
episode: 424 reward: -113.43178860591335
episode: 425 reward: -237.23038689470863
episode: 426 reward: -205.50622845546638
episode: 427 reward: -86.90556628710611
episode: 428 reward: -144.68937343312257
episode: 429 reward: -126.6551214526395
episode: 430 reward: -15

In [127]:
a = torch.tensor([[1,2],[3,4]])
b = torch.tensor([1,0], dtype=torch.long)
a[b]

a.gather(1, b.unsqueeze(1).long()).squeeze(1)

tensor([2, 3])

In [150]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class QNetwork(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)

    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)


In [151]:
import numpy as np
import random
from collections import namedtuple, deque

import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate 
UPDATE_EVERY = 4        # how often to update the network

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, seed):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.
        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # Compute Q targets for current states 
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local, self.qnetwork_target, TAU)                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)


class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)


In [154]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
env = gym.make('LunarLander-v2')
env.seed(0)
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)

def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores

agent = Agent(state_size=8, action_size=4, seed=0)
scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

/Users/epinyoanun/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
State shape:  (8,)
Number of actions:  4
Episode 100	Average Score: -225.00
Episode 200	Average Score: -172.18
Episode 272	Average Score: -107.51

KeyboardInterrupt: 